In [13]:
# BATCH_SIZE = 100
SEED = 61

import re
import time
import string
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
# import tensorflow_datasets as tfds
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Flatten
# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras import layers, Sequential

# from gensim.models.fasttext import FastText
# from gensim.models import Word2Vec

# import fasttext
# import fasttext.util

import torch
from transformers import AutoModel, AutoTokenizer

In [2]:
train_path = '../input/cleaned-sentiment-text/train.csv'#'data/cleaned/train.csv'
validation_path = '../input/cleaned-sentiment-text/validation.csv'#'data/cleaned/validation.csv'
test_path = '../input/cleaned-sentiment-text/test.csv'#'data/cleaned/test.csv'

In [3]:
# data is already cleaned and splitted in 3 parts: train, validation and test
train_df = pd.read_csv(train_path, index_col=0)
validation_df = pd.read_csv(validation_path, index_col=0)
test_df = pd.read_csv(test_path, index_col=0)
NUM_CLASSES = len(train_df['label'].unique())
index2class = {0:'neg', 1:'pos'}
class2index = {'neg': 0, 'pos':1}

# X
train_data = train_df.iloc[:, 0].to_numpy()
validation_data = validation_df.iloc[:, 0].to_numpy()
test_data = test_df.iloc[:, 0].to_numpy()
whole_data = np.concatenate((train_data, validation_data, test_data), dtype=object)

# y
label_train = np.array([class2index[i] for i in train_df.iloc[:, 1]])
label_validation = np.array([class2index[i] for i in validation_df.iloc[:, 1]])
label_test = np.array([class2index[i] for i in test_df.iloc[:, 1]])
label_whole = np.concatenate((label_train, label_validation, label_test))

print('Number of train data:', train_df.shape[0])
print('Number of validation data:', validation_df.shape[0])
print('Number of test data:', test_df.shape[0])
print('Number of classes:', NUM_CLASSES, '->', train_df['label'].unique())

MAXLEN = max([len(x.split()) for x in whole_data]) # = 492
EMBEDDING_DIM = 100

train_df.head()

Number of train data: 29744
Number of validation data: 9920
Number of test data: 9920
Number of classes: 2 -> ['neg' 'pos']


,text,label
0,hôm_nay đi ngang quán quyết_định ghé mua quán ...,neg
1,đến_súp lơ trưa nắng gắt chủ_nhật hy_vọng súp ...,pos
2,hôm_qua xe đông khiếp lo bàn may_sao bàn lầu l...,pos
3,món ăn_ở ngon món bò kho chủ quán quán phục_vụ...,pos
4,đẹp trời đi mỳ mỳ vô_cùng mặn sợi bở bàn phục_...,neg


In [4]:
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer
phobert, tokenizer = load_bert()

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
MAX_LENGTH = 256 # phobert default max sequence length
def phobert_tokenize(data):
    tokenized_data = []
    for line in data:
        tokenenized_line = tokenizer.encode(line, max_length=256, truncation=True)
        tokenized_data.append(tokenenized_line)
    return tokenized_data

In [28]:
# tokenize input sentence using phobert tokenizer
tokenized_train = phobert_tokenize(train_data)
tokenized_validation = phobert_tokenize(validation_data)
tokenized_test = phobert_tokenize(test_data)
# tokenized_whole = np.concatenate((tokenized_train, tokenized_validation, tokenized_test), dtype=object)

In [29]:
# pad sentence to a pre-defined max length
# no need truncating since it is already truncated in the phobert tokenizing process
padded_train = pad_sequences(tokenized_train, maxlen=MAXLEN, padding='post')
padded_validation = pad_sequences(tokenized_validation, maxlen=MAXLEN, padding='post')
padded_test = pad_sequences(tokenized_test, maxlen=MAXLEN, padding='post')

In [35]:
# np.array([[0.1, 0.15],
#                  [0.2, 0.25],
#                  [0.3, 0.35],
#                  [0.4, 0.45],
#                  [0.5, 0.55],
#                  [0.6, 0.65],
#                  [0.7, 0.75],
#                  [0.8, 0.85],
#                  [0.9, 0.95],
#                  [1.0, 1.5]]).reshape(10,2,1)

In [ ]:
# ref: https://miai.vn/2020/12/29/bert-series-chuong-3-thu-nhan-dien-cam-xuc-van-ban-tieng-viet-voi-phobert-cach-1/